In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pydicom

In [ ]:
import sys
sys.path.insert(0,'/content/drive/')
import os

# TODO: Fill in the Google Drive path where you uploaded the files 
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'Colab/final_project/'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

['COVID_combined', 'Normal', 'maskHE2_labels.csv', 'test', 'validation', 'ReadMe.gdoc', 'training.ipynb', 'training', 'train_dataset.npy', 'valid_dataset.npy', 'train_dataset_jiahong.npy', 'valid_dataset_jiahong.npy', 'Model_LungSegmentation', 'Model_Without_Lungseg', 'Copy of Siamese_Learning.ipynb', 'images_nomask', 'HE_output2.zip', 'Things to be done for Final Project.gdoc', 'Copy of Training_VGG16_CrossValidation_WithLungMasking_1.ipynb', 'Training_VGG16_CrossValidation_WithLungMasking.ipynb', 'Siamese_Learning.ipynb', 'grad_labels.csv', 'create_label.ipynb', 'Matching_code', 'val_labels.csv', 'test_labels.csv', 'train_labels.csv']


In [ ]:
import sys
sys.path.append(GOOGLE_DRIVE_PATH)

import time, os
os.environ["TZ"] = "US/Eastern"
time.tzset()

import numpy as np 
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
from skimage import io, transform
from PIL import Image

import copy

In [ ]:
training_transforms = transforms.Compose([
    transforms.Resize(336), # maybe better to factor out resize step
    transforms.CenterCrop(320),
    transforms.ToTensor()
])
validation_transforms = transforms.Compose([
    transforms.Resize(336), # maybe better to factor out resize step
    transforms.CenterCrop(320),
    transforms.ToTensor()
])

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import time
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

In [ ]:
train_csv = os.path.join(GOOGLE_DRIVE_PATH,'train_labels.csv')
val_csv = os.path.join(GOOGLE_DRIVE_PATH,'val_labels.csv')
test_csv = os.path.join(GOOGLE_DRIVE_PATH,'test_labels.csv')
grad_csv = os.path.join(GOOGLE_DRIVE_PATH,'grad_labels.csv')

class_numbers = 2

test_path = os.path.join(GOOGLE_DRIVE_PATH,"test")
train_path = os.path.join(GOOGLE_DRIVE_PATH,"training")
val_path = os.path.join(GOOGLE_DRIVE_PATH,"validation")

batch_size = 32

In [ ]:
# PyTorch modules
import torch
from torch import nn 
from torch.utils import data 
import torch.nn.functional as F 
from torchvision import transforms, models

# other modules
import os
import pandas as pd
import numpy as np
from PIL import Image
import random
import pydicom
import cv2
class CheXpert_Dataset(data.Dataset):
  
    def __init__(self, patient_table, image_dir, epoch_size, transform=None):
 
        self.patient_table = patient_table
        self.image_dir = image_dir # note, images are already 8-bit, full-sized JPEGs
        self.transform = transform
        self.epoch_size = epoch_size 
        if self.transform is None:
            self.transform = transforms.Compose([transforms.ToTensor()])
 
    def __len__(self):
        return self.epoch_size
 
    def __getitem__(self, idx): 
        
        patient_table = self.patient_table
        # separate tables for abnormal lung labels 0 and 1
        table_A = patient_table[patient_table['encoded_labels'] == 0]
        table_B = patient_table[patient_table['encoded_labels'] == 1]

        # goal is 50:50 distribution of change vs no change
        change_binary = random.randint(0,1) 

        # if no change 
        if change_binary == 0:

            # select two random images 
            table_A_or_B = random.randint(0,1)

            if table_A_or_B == 0:
                pick0 = random.choice(range(len(table_A)))
                pick1 = random.choice(range(len(table_A)))
                img0 = Image.open(self.image_dir + '/' + table_A.iloc[pick0]['Images'])
                img1 = Image.open(self.image_dir + '/' + table_A.iloc[pick1]['Images'])

                abnormal_lung_0 = 0
                abnormal_lung_1 = 0

            if table_A_or_B == 1:
                pick0 = random.choice(range(len(table_B)))
                pick1 = random.choice(range(len(table_B)))
                img0 = Image.open(self.image_dir + '/' + table_B.iloc[pick0]['Images'])
                img1 = Image.open(self.image_dir + '/' + table_B.iloc[pick1]['Images'])

                abnormal_lung_0 = 1
                abnormal_lung_1 = 1

        # if change -- note: direction of change does not matter in this step
        if change_binary == 1: 

            pick0 = random.choice(range(len(table_A)))
            pick1 = random.choice(range(len(table_B)))
            img0 = Image.open(self.image_dir + '/'+ table_A.iloc[pick0]['Images'])
            img1 = Image.open(self.image_dir + '/' + table_B.iloc[pick1]['Images'])

            abnormal_lung_0 = 0
            abnormal_lung_1 = 1

        # 0 for no change, 1 for change (difference between labels)
        if abnormal_lung_0 == abnormal_lung_1:
            change_label = 0
        else:
            change_label = 1
 
        meta = {"path0": patient_table.iloc[pick0]['Images'],
                "path1": patient_table.iloc[pick1]['Images'],
                "abnormal_lung_0": abnormal_lung_0,
                "abnormal_lung_1": abnormal_lung_1,
                }

        if self.transform is not None:
            img0 = self.transform(img0)
            img1 = self.transform(img1) 

        return img0, img1, change_label, meta

In [ ]:
class SiameseNetwork_DenseNet121(nn.Module):

    def __init__(self):
        super(SiameseNetwork_DenseNet121, self).__init__()
        self.cnn0 = torch.hub.load('pytorch/vision:v0.5.0', 'densenet121', pretrained=True)
        # map last layer from 1000 node outcome to three nodes
        #self.cnn0.fc = nn.Linear(1000, 3) # mapping input image to a 3D space

    def forward_once(self, x):
        output = self.cnn0(x)
        return output

    def forward(self, input0, input1):
        output0 = self.forward_once(input0)
        output1 = self.forward_once(input1)
        return output0, output1
  

In [ ]:
class ContrastiveLoss(torch.nn.Module):
    """
    Contrastive loss function.
    Based on: http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    Modified from: https://hackernoon.com/facial-similarity-with-siamese-networks-in-pytorch-9642aa9db2f7
    """ 

    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output0, output1, label):
        euclidean_distance = F.pairwise_distance(output0, output1)
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))

        return loss_contrastive

In [ ]:
class MSELoss(torch.nn.Module):
    """
    MSE loss
    """ 

    def __init__(self):
        super(MSELoss, self).__init__()

    def forward(self, output0, output1, label):
        euclidean_distance = F.pairwise_distance(output0, output1)
    
        return F.mse_loss(euclidean_distance,label)

In [ ]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
import torchvision
from torchvision import transforms, models
from torch.autograd import Variable
  
# other modules 
import os
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import statistics
import pickle
from tqdm import tqdm
#working_path = '/content/gdrive/My Drive/EECS545/'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
training_table = pd.read_csv(train_csv)
validation_table = pd.read_csv(val_csv)
table_A = validation_table[validation_table['encoded_labels'] == 1]

In [ ]:
training_siamese_dataset = CheXpert_Dataset(patient_table = training_table, 
                                        image_dir = train_path, 
                                        epoch_size = 1000,
                                        transform = training_transforms)
 
training_dataloader = torch.utils.data.DataLoader(training_siamese_dataset, 
                                                  batch_size=8, 
                                                  shuffle=True, 
                                                  num_workers=1)

In [ ]:
validation_siamese_dataset = CheXpert_Dataset(patient_table = validation_table, 
                                        image_dir = val_path, 
                                        epoch_size = 1000,
                                        transform = validation_transforms)
 
validation_dataloader = torch.utils.data.DataLoader(validation_siamese_dataset, 
                                                  batch_size=8, 
                                                  shuffle=True, 
                                                  num_workers=1)
import os.path
from os import path

**Grad-CAM**

In [ ]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
import torchvision
from torchvision import transforms, models
from torch.autograd import Variable
  
# other modules 
import os
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import statistics
import pickle
from tqdm import tqdm
from PIL import Image
import seaborn as sns
import scipy
from sklearn import metrics
#output_dir = '/content/drive/MyDrive/EECS545/matching_results'
net = SiameseNetwork_DenseNet121().cuda()
net.load_state_dict(torch.load(GOOGLE_DRIVE_PATH+'Matching_code/Sara_PXS_score_model_epoch1000_iter110.pth'))
def img_processing(input_image):
    '''
    processes PIL image file
    '''
    
    transf = transforms.Compose([
        transforms.Resize(336), # maybe better to factor out resize step
        transforms.CenterCrop(320),
        transforms.ToTensor()
    ])

    output_image= transf(input_image)
    output_image = np.repeat(output_image, 3, 0)
    output_image= output_image[np.newaxis, ...]
    output_image = Variable(output_image).cuda()

    return output_image
def anchor_inference(img_anchor, image_path, net):
    '''
    takes img_anchor list object, image path of interest, and siamese network
    '''

    img_comparison = img_processing(Image.open(image_path))

    save_euclidean_distance = []
    net.eval()
    for j in range(len(img_anchor)):
        output1, output2 = net.forward(img_anchor[j], img_comparison)
        euclidean_distance = F.pairwise_distance(output1, output2)
        save_euclidean_distance.append(euclidean_distance.item())

    # take median euclidean distance compared to the the pool of normals
    return statistics.mean(save_euclidean_distance)

print(net)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.5.0


SiameseNetwork_DenseNet121(
  (cnn0): DenseNet(
    (features): Sequential(
      (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu0): ReLU(inplace=True)
      (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (denseblock1): _DenseBlock(
        (denselayer1): _DenseLayer(
          (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace=True)
          (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU(inplace=True)
          (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
        (denselayer2): _DenseLayer(
          (norm1): BatchNorm2d(96, eps=1e-

In [ ]:
training_table = pd.read_csv(train_csv)
table_A = training_table[training_table['encoded_labels'] == 1].sample(n=100)
table_B = training_table[training_table['encoded_labels'] == 0].sample(n=100)

In [ ]:
test_table = pd.read_csv(grad_csv)

In [ ]:
img_anchor_1 = []
for a in tqdm(range(len(table_A))):
    image_path = train_path+'/' + table_A.iloc[a]['Images']
    img_anchor_1.append(img_processing(Image.open(image_path)))

100%|██████████| 100/100 [00:28<00:00,  3.46it/s]


In [ ]:
img_anchor_0 = []
for a in tqdm(range(len(table_B))):
    image_path = train_path+'/' + table_B.iloc[a]['Images']
    img_anchor_0.append(img_processing(Image.open(image_path)))

100%|██████████| 100/100 [00:31<00:00,  3.19it/s]


In [ ]:
# Extract  pretrained activations
class SaveFeatures():
    """ Extract pretrained activations"""
    features=None
    def __init__(self, m):
        self.hook = m.register_forward_hook(self.hook_fn)
    def hook_fn(self, module, input, output):
        self.features = ((output.cpu()).data).numpy()
    def remove(self):
        self.hook.remove()
  
class SaveGradients():
    features=None
    def __init__(self, m):
        self.hook = m.register_backward_hook(self.hook_fn)
    def hook_fn(self, module, grad_input, grad_output):
        self.features = ((grad_output[0].detach().cpu()).data).numpy()
    def remove(self):
        self.hook.remove()
print(type(net))
final_layer = net.cnn0.features.norm5
activated_features = SaveFeatures(final_layer)
activated_gradients = SaveGradients(final_layer)

<class '__main__.SiameseNetwork_DenseNet121'>


In [ ]:
def cam_img(feature_map, grads):
    _, nc, h, w = feature_map.shape
    cam = np.zeros((h,w), dtype=np.float32)	
    #print(cam.shape)
    grads = grads[0]
    #print(grads.shape)	
    weights = np.mean(grads, axis=(1,2))
    #print(weights.shape)
    for i, w in enumerate(weights):
        cam += w * feature_map[0, i, :, :]
    #print(cam.shape)
    cam = np.maximum(cam, 0)
    cam = cam / cam.max()

    return cam

In [ ]:
proba = []
heatmaps = []
original_images = []
predict_labels = []
gt_labels = []

for a in tqdm(range(len(test_table))):
  image_path = test_path+'/' + test_table.iloc[a]['Images']

  #forward:
  PXS_0 = anchor_inference(img_anchor_0, image_path, net)
  PXS_1 = anchor_inference(img_anchor_1, image_path, net)

  if PXS_1>PXS_0:
     pred_idx=0
     class_loss = PXS_1
  else:
     pred_idx=1
     class_loss = PXS_0

  predict_labels.append(pred_idx)
  gt_labels.append(test_table.iloc[a]['encoded_labels'])

  cur_image = np.asarray(Image.open(image_path))#.transpose((0, 2, 3, 1))
  print(cur_image.shape)
  #print(cur_image.shape)
  original_images.append(cur_image)

  # backward
  net.zero_grad()
  class_loss.backward()

  print(activated_gradients.features)
  print(activated_features.features)
  #print(activated_features.features.shape)
  print(activated_gradients.features.shape)

  grad_img = cam_img(activated_features.features, activated_gradients.features)
  #activated_features.remove()
  heatmaps.append(grad_img)
    
print(len(proba), len(heatmaps),len(predict_labels))
print(proba)
print(predict_labels)
print(gt_labels)

  0%|          | 0/8 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:795: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
  0%|          | 0/8 [00:06<?, ?it/s]

(512, 512)


AttributeError: ignored

In [ ]:
correct=0
for a in tqdm(range(len(test_table))):
   image_path = test_path+'/' + test_table.iloc[a]['Images']
   PXS_0 = anchor_inference(img_anchor_0,image_path, net)
   PXS_1 = anchor_inference(img_anchor_1,image_path, net)
   if PXS_1>PXS_0:
     label=0
   else:
     label=1
   if label==test_table.iloc[a]['encoded_labels']:
     correct+=1
